In [3]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path
import platform
import tkinter as tk
from tkinter import filedialog

from neuropy.io.neuroscopeio import NeuroscopeIO
from neuropy.io.binarysignalio import BinarysignalIO 
from neuropy.io.miniscopeio import MiniscopeIO
from neuropy.core import Epoch
from neuropy.utils import plot_util
from neuropy.plotting.spikes import plot_raster
from neuropy.plotting.signals import plot_signal_w_epochs




sys.path.insert(1, '/content/cnn-ripple/src/cnn/');


In [4]:
# Define a class for a typical recording or set of recordings
class ProcessData:
    def __init__(self, basepath):
        basepath = Path(basepath)
        self.basepath = basepath
        xml_files = sorted(basepath.glob("*.xml"))
        assert len(xml_files) == 1, "Found more/less than one .xml file"
        
        fp = xml_files[0].with_suffix("")
        self.filePrefix = fp
        
        self.recinfo = NeuroscopeIO(xml_files[0])
        eegfiles = sorted(basepath.glob('*.eeg'))
        assert len(eegfiles) == 1, "Fewer/more than one .eeg file detected"
        self.eegfile = BinarysignalIO(eegfiles[0], n_channels=self.recinfo.n_channels,
                                     sampling_rate=self.recinfo.eeg_sampling_rate,
                                     )
        try:
            self.datfile = BinarysignalIO(eegfiles[0].with_suffix('.dat'),
                                         n_channels=self.recinfo.n_channels,
                                         sampling_rate=self.recinfo.dat_sampling_rate,
                                         )
        except FileNotFoundError:
            print('No dat file found, not loading')
                
        
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.recinfo.source_file.name})"
    
def sess_use(basepath=os.getcwd()):
    """Load in data. Uses current directory as default"""

    return ProcessData(basepath)

In [7]:
# Open a directory chooser dialog
dir_use = filedialog.askdirectory(title="Please select a data folder")

# Check if user selected a directory or pressed cancel
if dir_use:
    print(f"Selected Data Directory: {dir_use}")
else:
    print("No directory was selected.")



# dir_use = '/home/kimqi/Documents/Data/Orange/20230905'  # Add directory here!
sess = sess_use(dir_use)
print(sess.recinfo)
print(sess.eegfile)

Selected Data Directory: D:/Data/RippleDetection/20230309Recall1
No dat file found, not loading
filename: D:\Data\RippleDetection\20230309Recall1\Django_recall1_denoised.xml 
# channels: 134
sampling rate: 30000
lfp Srate (downsampled): 1250

duration: 4532.47 seconds 
duration: 1.26 hours 


In [8]:
from neuropy.analyses.artifact import detect_artifact_epochs
signal = sess.eegfile.get_signal()
buffer_add = 0.1  # seconds, None = don't add

art_epochs_file = sess.filePrefix.with_suffix(".art_epochs.npy")
if art_epochs_file.exists():
    art_epochs = Epoch(epochs=None, file=art_epochs_file)
    print('Existing artifact epochs file loaded')
else:
    art_epochs = detect_artifact_epochs(signal, thresh=6, 
                                    edge_cutoff=1, merge=6)

    if buffer_add is not None:  # Add in buffer to prevent erroneous detection of start/stop of artifact as SWRs
        art_epochs.add_epoch_buffer(buffer_add)
    sess.recinfo.write_epochs(epochs=art_epochs, ext='art')  # Write to neuroscope
    art_epochs.save(art_epochs_file)
art_epochs

Buffer of 0.1 added before/after each epoch
D:\Data\RippleDetection\20230309Recall1\Django_recall1_denoised.art_epochs.npy saved


50 epochs
Snippet: 
       start      stop label
0   15.8128   16.0408      
1  146.7296  147.0024      
2  270.5160  270.7768      
3  410.8512  411.0968      
4  421.0152  421.3176      

In [9]:
print(signal)

In [11]:
# Debugging Cell
%whos

Variable                        Type           Data/Info
--------------------------------------------------------
BinarysignalIO                  type           <class 'neuropy.io.binary<...>signalio.BinarysignalIO'>
Epoch                           type           <class 'neuropy.core.epoch.Epoch'>
MiniscopeIO                     type           <class 'neuropy.io.miniscopeio.MiniscopeIO'>
NeuroscopeIO                    type           <class 'neuropy.io.neuroscopeio.NeuroscopeIO'>
Path                            type           <class 'pathlib.Path'>
ProcessData                     type           <class '__main__.ProcessData'>
art_epochs                      Epoch          <object with id 1283194176848 (str() failed)>
art_epochs_file                 WindowsPath    D:\Data\RippleDetection\2<...>1_denoised.art_epochs.npy
buffer_add                      float          0.1
detect_artifact_epochs          function       <function detect_artifact<...>hs at 0x0000012AC3F61D00>
dir_path         